In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
mapp = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)

pmapp = mapp[mapp.biotype == 'protein_coding'].copy()

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
outs = pd.read_csv('Supp_table_all_variants.SVs_included.csv')

In [3]:
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

def go_overrepresentation(genes_of_interest, reference_genes):
    obo = pd.read_csv('genes2go.tsv', sep='\t')

    go2des = obo[['go', 'description']].copy()
    go2des = go2des.drop_duplicates('go').copy()
    go2des = go2des.set_index('go', drop=False)
    
    # keep only those in reference genes
    obo = obo[obo['genes'].isin(reference_genes)]
    obo_genes = list(obo['genes'].unique())
    num_total_genes = len(obo_genes)

    # keep only those with annotations
    inn = []
    for s in genes_of_interest:
        if s in obo_genes:
            inn.append(s)
    genes_of_interest = inn
    
    # create dict go of terms to genes
    go_dict = {}
    for go_term, df in obo.groupby('go'):
        go_dict[go_term] = df.genes.to_list()
        
    
    output = []
    for go_term in go_dict.keys():    
        go_genes = go_dict[go_term]
        num_genes_in_go = len(go_genes)
        num_genes_not_in_go = num_total_genes - num_genes_in_go

        goi_and_go = 0
        goi_and_not_go = 0
        for s in genes_of_interest:
            if s in go_genes:
                goi_and_go = goi_and_go + 1
            else:
                goi_and_not_go = goi_and_not_go + 1

        not_goi_and_go = num_genes_in_go - goi_and_go
        not_goi_and_not_go = num_genes_not_in_go - goi_and_not_go
        oddsratio, pvalue = fisher_exact([[goi_and_go, goi_and_not_go], [not_goi_and_go, not_goi_and_not_go]])

        to_append = [go_term, goi_and_go, goi_and_not_go, not_goi_and_go, not_goi_and_not_go, oddsratio, pvalue]
        output.append(to_append)
        
    out = pd.DataFrame(output)
    out = out.sort_values(6)
    out = out.set_index(0, drop=False)
    out['des'] = go2des.loc[out[0], 'description']
    out['FDR'] = multipletests(out[6], method='fdr_bh')[1]
    return out    

In [7]:
net = pd.read_csv('output/{}_nearest_neighbor_weighted.txt'.format('UQCRC2'), sep='\t')
net = net.set_index('End gene', drop=False)
end_genes = net['End gene'].to_list()
ref = end_genes

In [33]:
for sub in outs['sample'].unique():
    with open('cohort_connectors/{}.outlier_and_rare_variant_connectors.txt'.format(sub), 'r') as f:
        genes = f.readlines()
    genes = [s.strip() for s in genes]
    go = goslim_overrepresentation(genes, ref)

    print(sub, go[go.FDR < 0.5].shape)

        
    go.to_csv('cohort_connectors/{}.outlier_and_rare_variant_connectors_goslim.txt'.format(sub), sep='\t', index=False)

SG001 (755, 9)
SG003 (198, 9)
SG006 (883, 9)
SG007 (1019, 9)
SG002 (1186, 9)
SG011 (491, 9)
SG025 (600, 9)
SG022 (518, 9)
SG024 (665, 9)
SG021 (899, 9)
SG023 (756, 9)
SG026 (727, 9)
SG155 (893, 9)
SG030 (739, 9)
SG027 (826, 9)
SG031 (1026, 9)
SG069 (1131, 9)
SG040 (906, 9)
SG039 (1145, 9)
SG038 (956, 9)
SG037 (934, 9)
SG045 (1072, 9)
SG046 (833, 9)
SG041 (560, 9)
SG042 (785, 9)
SG043 (1171, 9)
SG044 (1086, 9)
SG151 (1062, 9)
SG149 (919, 9)
SG148 (851, 9)
SG152 (1106, 9)
SG150 (1154, 9)
